In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import xgboost as xgb
from xgboost import plot_importance, plot_tree

import utility_functions as fn

In [2]:
energy = pd.read_csv('energy_clean.csv',index_col=0)
energy.index = pd.to_datetime(energy.index,format='%Y-%m-%d %H:00:00')

energy['Yale_kWh'] = energy['YUAG']+energy['Berkeley']+energy['Hopper']+energy['17HH']
energy = pd.DataFrame(energy['Yale_kWh'])

In [3]:
weather = pd.read_csv('weather_clean.csv',index_col=0)
weather.index = pd.to_datetime(weather.index,format='%Y-%m-%d %H:00:00')

weather = fn.add_hours_before(weather,np.arange(1,24))
weather = weather.loc[energy.index[0]:energy.index[len(energy)-1]]

# weather.drop('2018-03-11 02:00:00',inplace=True)

In [4]:
split_date = '2018-07-26 00:00:00'

# train = energy.loc[energy.index <= split_date].copy()
# test = energy.loc[energy.index > split_date].copy()

#X_train, y_train = fn.date_features(train, label='Yale_kWh')
#X_test, y_test = fn.date_features(test, label='Yale_kWh')

In [48]:
X_train = weather.loc[weather.index <= split_date].copy()
X_test = weather.loc[weather.index > split_date].copy()

y_train = energy.loc[energy.index <= split_date].copy()
y_test = energy.loc[energy.index > split_date].copy()

# X_train.index.difference(y_train.index)

In [60]:
# X_train.index.difference(y_train.index)

X_train.loc['2018-03-11 00:00:00':'2018-03-11 05:00:00']

y_train.append({'2018-03-11 02:00:00':400},ignore_index=True,sort=True)

y_train.loc['2018-03-11 00:00:00':'2018-03-11 05:00:00']

,Yale_kWh
2018-03-11 00:00:00,432.2
2018-03-11 01:00:00,372.3
2018-03-11 03:00:00,423.1
2018-03-11 04:00:00,400.4
2018-03-11 05:00:00,396.8
2018-03-11 02:00:00,400.0


In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
        verbose=False)

In [ ]:
_ = plot_importance(reg, height=0.9)

In [ ]:
test['Prediction_kWh'] = reg.predict(X_test)
yale_all = pd.concat([test, train], sort=False)
_ = yale_all[['Yale_kWh','Prediction_kWh']].plot(figsize=(15, 5))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_true=test['Yale_kWh'], y_pred=test['Prediction_kWh'])